In [1]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

# Set the theme once
pio.templates.default = "plotly_white"

from util import multi_select_counts, get_subplot_coords, df_prop, df_multiselect_prop
import mappings

In [2]:
df = pl.read_csv("OriginalData/Preprogram_Survey.csv")
df = df.with_columns(
    pl.col("What is your current education status?").map_elements(lambda x: "True" if ("Currently" in x) else "False", return_dtype=pl.String).alias("College Student"),
    pl.col("What is your gender identity?").map_elements(lambda x: "Female" if ("Female" == x) else "Not Female", return_dtype=pl.String).alias("Gender"),
    pl.lit(1).alias("Count") 
)

In [3]:
df_prop(df, 'What is your current education status?', 100)

What is your current education status?
shape: (6, 3)
┌─────────────────────────────────┬─────┬──────┐
│ What is your current education… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ High school graduate* (not in … ┆ 24  ┆ 0.24 │
│ Recently graduated from colleg… ┆ 8   ┆ 0.08 │
│ Enrolled in a trade or apprent… ┆ 5   ┆ 0.05 │
│ null                            ┆ 3   ┆ 0.03 │
│ Currently enrolled in 2/4-year… ┆ 53  ┆ 0.53 │
│ In high school                  ┆ 7   ┆ 0.07 │
└─────────────────────────────────┴─────┴──────┘


In [4]:
df_prop(df, 'What is your gender identity?', 100)

What is your gender identity?
shape: (5, 3)
┌───────────────────────────────┬─────┬──────┐
│ What is your gender identity? ┆ len ┆ prop │
│ ---                           ┆ --- ┆ ---  │
│ str                           ┆ u32 ┆ f64  │
╞═══════════════════════════════╪═════╪══════╡
│ Female                        ┆ 57  ┆ 0.57 │
│ Non-binary                    ┆ 2   ┆ 0.02 │
│ Male                          ┆ 39  ┆ 0.39 │
│ null                          ┆ 1   ┆ 0.01 │
│ Prefer not to answer          ┆ 1   ┆ 0.01 │
└───────────────────────────────┴─────┴──────┘


In [5]:
fig = px.histogram(df, x="Gender", y="Count",
             color='College Student', barmode='group', 
             height=400, width = 600, text_auto=True, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textfont_size=12, textposition="inside")
fig.update_layout(yaxis_title="Number of Participants")
fig.show()

In [6]:
df_prop(df, 'Have you held a paid job before?', 100)

Have you held a paid job before?
shape: (2, 3)
┌─────────────────────────────────┬─────┬──────┐
│ Have you held a paid job befor… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ Yes                             ┆ 82  ┆ 0.82 │
│ No                              ┆ 18  ┆ 0.18 │
└─────────────────────────────────┴─────┴──────┘


In [7]:
df_prop(df, 'What is your gender identity?', 100)

What is your gender identity?
shape: (5, 3)
┌───────────────────────────────┬─────┬──────┐
│ What is your gender identity? ┆ len ┆ prop │
│ ---                           ┆ --- ┆ ---  │
│ str                           ┆ u32 ┆ f64  │
╞═══════════════════════════════╪═════╪══════╡
│ Prefer not to answer          ┆ 1   ┆ 0.01 │
│ null                          ┆ 1   ┆ 0.01 │
│ Female                        ┆ 57  ┆ 0.57 │
│ Non-binary                    ┆ 2   ┆ 0.02 │
│ Male                          ┆ 39  ┆ 0.39 │
└───────────────────────────────┴─────┴──────┘


In [8]:
df_multiselect_prop(df, ["Hispanic or Latino", 'White', 'Asian or Pacific Islander', 'American Indian or Alaska Native', 'Black or African American'], 100)

,value,count,prop
0,Black or African American,90,0.90
1,Hispanic or Latino,12,0.12
2,White,2,0.02
3,Asian or Pacific Islander,1,0.01


In [9]:
df_prop(df, 'Have you held a paid job before?', 100)

Have you held a paid job before?
shape: (2, 3)
┌─────────────────────────────────┬─────┬──────┐
│ Have you held a paid job befor… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ Yes                             ┆ 82  ┆ 0.82 │
│ No                              ┆ 18  ┆ 0.18 │
└─────────────────────────────────┴─────┴──────┘


In [10]:
df_multiselect_prop(df, mappings.prev_job_col[1:], 100)

,value,count,prop
0,Internship / fellowship,38,0.38
1,Retail / customer service,37,0.37
2,Food service / hospitality,29,0.29
3,Office or clerical work,26,0.26
4,Youth mentoring or childcare,21,0.21
5,Family business / informal work,7,0.07


In [11]:
prev_job = multi_select_counts(df, mappings.prev_job_col, mappings.prev_job_map)

# Create standalone bar chart
fig = go.Figure(
    data=[
        go.Bar(
            x=prev_job.index,
            y=prev_job.values,
            name="Previous Job Experience",
            marker=dict(color='darkblue')
        )
    ]
)

fig.update_layout(
    title="Previous Job Experience",
    xaxis_title="Job Type",
    yaxis_title="Count",
    template="plotly_white"
)

fig.show()

In [12]:
df_multiselect_prop(df, ['Build professional experience',
 'Improve job readiness and soft skills',
 'Learn about career options',
 'Grow my resume or LinkedIn profile',
 'Network with professionals',
 'Gain mentorship and guidance',
 'Improve my financial knowledge'], 100)

,value,count,prop
0,Build professional experience,68,0.68
1,Improve job readiness and soft skills,43,0.43
2,Grow my resume or LinkedIn profile,39,0.39
3,Network with professionals,33,0.33
4,Gain mentorship and guidance,29,0.29
5,Improve my financial knowledge,20,0.20
6,Learn about career options,13,0.13


In [13]:
goal = multi_select_counts(df, ['#','Build professional experience',
 'Improve job readiness and soft skills',
 'Learn about career options',
 'Grow my resume or LinkedIn profile',
 'Network with professionals',
 'Gain mentorship and guidance',
 'Improve my financial knowledge'])

# Create standalone bar chart
fig = go.Figure(
    data=[
        go.Bar(
            x=goal.index,
            y=goal.values,
            name="Goal of Participation",
            
        )
    ]
)

fig.update_layout(
    title="Goal of Participation",
    xaxis_title="Identified Goal",
    yaxis_title="Count",
    template="plotly_white"
)

fig.show()

In [14]:
career = multi_select_counts(df, mappings.career_consideration_col, mappings.career_consideration_map)

fig = go.Figure(
    data=[
        go.Pie(
            labels=career.index,
            values=career.values,
            hole=0.4,  # Set to 0.4 for a donut chart
            textinfo='percent+label',  # Options: 'label', 'value', 'percent', 'label+percent'
        )
    ]
)

fig.update_layout(
    title="Key Drivers of Career Decisions",
    template="plotly_white",
    width = 800,
    height = 500
)

fig.show()

In [15]:
dev_needs = multi_select_counts(df, mappings.help_needed_col)

fig = go.Figure(
    data=[
        go.Pie(
            labels=dev_needs.index,
            values=dev_needs.values,
            hole=0.4,  # Set to 0.4 for a donut chart
            textinfo='percent+label',  # Options: 'label', 'value', 'percent', 'label+percent'
        )
    ]
)

fig.update_layout(
    title="Development Needs",
    template="plotly_white",
    width = 800,
    height = 500
)

fig.show()

In [16]:
jobs_desired = {
    'Healthcare & Wellness': 35,
    'Business & Finance': 28,
    'Technology & IT': 15,
    'Law & Public Safety': 14,
    'Education & Human Services': 13,
    'Creative Arts & Media': 10,
    'Trades & Skilled Labor': 8,
    'Science & Research (General)': 5,
    'Other/Vague': 4
}

df_job = pl.DataFrame({"Job": jobs_desired.keys(), "count": jobs_desired.values()})

fig = go.Figure(
    data=[
        go.Pie(
            labels=df_job["Job"],
            values=df_job["count"],
            hole=0.4,  # Set to 0.4 for a donut chart
            textinfo='percent+label',  # Options: 'label', 'value', 'percent', 'label+percent'
        )
    ]
)

fig.update_layout(
    title="Job Interest Breakdown by Career Category",
    template="plotly_white",
    width = 800,
    height = 500
)

fig.show()

In [17]:
sc = df.select(mappings.score_card + ["College Student", "Gender", "Count"])
sc = sc.with_columns([
    pl.col(col).replace(mappings.entry_standardization_mapping).replace(mappings.skill_rating_map).cast(pl.Int32).alias(col)
    for col in mappings.score_card
])

r_map = {v:k for k,v in mappings.score_card_colname_name_mapping.items()}

sc = sc.unpivot(on=mappings.score_card, index=["College Student", "Gender", "Count"], value_name="Rating", variable_name="Question").with_columns(pl.col("Question").replace(r_map))

In [18]:
fig = px.histogram(sc, x="Question", y="Rating",
             color='College Student', barmode='group', 
             height=400, histfunc = "avg")
fig.update_traces(textfont_size=12, textposition="inside")
fig.update_layout(yaxis_title="Mean Rating")
fig.show()

In [19]:
for s in set(mappings.score_card + [ 'I know how to present myself professionally in different settings',
 'I believe in my ability to grow and improve through effort',
 'I have a support system (family, friends, etc.) that encourages my success',
 'I can manage challenges without giving up',
 'I am clear about the type of person I want to become',
 'Motivated to work toward your goals',
 'Focused and productive',
 'Stressed or overwhelmed',
 'Confident about your future',
 'Supported by others']):
    print(s)
    print(df_prop(df, s, 100))
    print()

I can manage challenges without giving up
I can manage challenges without giving up
shape: (6, 3)
┌─────────────────────────────────┬─────┬──────┐
│ I can manage challenges withou… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ Strongly agree                  ┆ 31  ┆ 0.31 │
│ Neutral                         ┆ 20  ┆ 0.2  │
│ null                            ┆ 1   ┆ 0.01 │
│ Strongly disagree               ┆ 2   ┆ 0.02 │
│ Agree                           ┆ 45  ┆ 0.45 │
│ Disagree                        ┆ 1   ┆ 0.01 │
└─────────────────────────────────┴─────┴──────┘
None

I believe in my ability to grow and improve through effort
I believe in my ability to grow and improve through effort
shape: (5, 3)
┌─────────────────────────────────┬─────┬──────┐
│ I believe in my ability to gro… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                        

In [20]:
df_prop(df, "How often do you save money *(beyond daily spending)*?", 100)

How often do you save money *(beyond daily spending)*?
shape: (4, 3)
┌─────────────────────────────────┬─────┬──────┐
│ How often do you save money *(… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ I try to save consistently      ┆ 28  ┆ 0.28 │
│ I save with a specific financi… ┆ 28  ┆ 0.28 │
│ Rarely or never                 ┆ 5   ┆ 0.05 │
│ Sometimes, when I can           ┆ 39  ┆ 0.39 │
└─────────────────────────────────┴─────┴──────┘


In [21]:
fig = px.histogram(df, x="How often do you save money *(beyond daily spending)*?", y="Count",
             color='College Student', barmode='group', 
             height=400, width = 800, text_auto=True, color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.update_traces(textfont_size=14, textposition="outside")
fig.update_layout(yaxis_title="Number of Participants", yaxis_range=[0, 25])
fig.show()

In [22]:
financial_literacy_col = ['Creating a personal budget',
 'Understanding credit and credit scores',
 'Opening and managing a bank account',
 'Differences between checking/savings/investment accounts',
 'Using financial apps or online banking tools',
 'Investing your money for future wealth']

In [31]:
for f in financial_literacy_col:
    print(f)
    print(df_prop(df, f, 100))
    print()

Creating a personal budget
Creating a personal budget
shape: (5, 3)
┌────────────────────────────┬─────┬──────┐
│ Creating a personal budget ┆ len ┆ prop │
│ ---                        ┆ --- ┆ ---  │
│ str                        ┆ u32 ┆ f64  │
╞════════════════════════════╪═════╪══════╡
│ Somewhat                   ┆ 25  ┆ 0.25 │
│ Extremely                  ┆ 21  ┆ 0.21 │
│ Very                       ┆ 37  ┆ 0.37 │
│ Not familiar               ┆ 6   ┆ 0.06 │
│ Slightly                   ┆ 11  ┆ 0.11 │
└────────────────────────────┴─────┴──────┘
None

Understanding credit and credit scores
Understanding credit and credit scores
shape: (5, 3)
┌─────────────────────────────────┬─────┬──────┐
│ Understanding credit and credi… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ Slightly                        ┆ 21  ┆ 0.21 │
│ Very                            ┆ 18  ┆ 0.18 │
│ Somew

In [24]:
df_prop(df, 'Do you currently have a bank account?', 100)

Do you currently have a bank account?
shape: (4, 3)
┌─────────────────────────────────┬─────┬──────┐
│ Do you currently have a bank a… ┆ len ┆ prop │
│ ---                             ┆ --- ┆ ---  │
│ str                             ┆ u32 ┆ f64  │
╞═════════════════════════════════╪═════╪══════╡
│ I’m not sure                    ┆ 1   ┆ 0.01 │
│ No                              ┆ 6   ┆ 0.06 │
│ Yes, but no debit card          ┆ 1   ┆ 0.01 │
│ Yes, with a debit card          ┆ 92  ┆ 0.92 │
└─────────────────────────────────┴─────┴──────┘


In [25]:
financial_tools = multi_select_counts(df, ["#" ,'Online/mobile banking app', 'Cash App / Venmo / Zelle / PayPal', 'Savings account',
 'Budgeting app or spreadsheet',
 'None of the above',
 'Robinhood/Fidelity/Acorns (Investment Apps)'])

In [26]:
financial_tools

value
Cash App / Venmo / Zelle / PayPal              83
Online/mobile banking app                      78
Savings account                                42
Robinhood/Fidelity/Acorns (Investment Apps)    10
Budgeting app or spreadsheet                    4
Name: count, dtype: int64

In [32]:
fig = go.Figure(
    data=[
        go.Pie(
            labels=financial_tools.index,
            values=financial_tools.values,
            hole=0.4,  # Set to 0.4 for a donut chart
            textinfo='percent+label',  # Options: 'label', 'value', 'percent', 'label+percent',
            marker_colors=px.colors.qualitative.Pastel
        )
    ]
)

fig.update_layout(
    title="Financial Services Used",
    template="plotly_white",
    width = 800,
    height = 500
)

fig.show()

In [28]:
fig = make_subplots(rows=2, cols=4,
                    subplot_titles=list(mappings.score_card_colname_name_mapping.keys()))

sc = df.select(["#"] + mappings.score_card).to_pandas()
sc = pl.DataFrame(sc.replace(mappings.entry_standardization_mapping))

for index, s in enumerate(mappings.score_card):
    
    position = get_subplot_coords(index, 2, 4)
    
    d = multi_select_counts(sc, ["#", s])
    fig.add_trace(
        go.Bar(y=d.values, x=d.index),
        row=position[0], col=position[1])

In [29]:
# Update layout
fig.update_layout(height=600, width=1000, title_text="Baseline Self-Assessment of Personal and Professional Skills")
fig.update_layout(showlegend=False)
fig.update_xaxes(
    categoryorder='array',
    categoryarray=['Not at all', 'Slightly', 'Moderately','Very', 'Extremely'])
fig.show()

In [30]:
df_numeric = sc[mappings.skill_cols].to_pandas().replace(mappings.skill_rating_map)

# Step 3: Average across all responses per skill
avg_scores = df_numeric.mean()

# Step 4: Plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=avg_scores.tolist(),
    theta=avg_scores.index.tolist(),
    fill='toself',
    name='Average Scores'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[1, 5],
            tickvals=[1, 5],  
            ticktext=["Low", "High"]  
        )
    ),
    title="Overview Self-Ratings Across Key Skills",
    showlegend=False
)

fig.show()

C:\Users\justi\AppData\Local\Temp\ipykernel_9372\352457242.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

